In [1]:
import os, inspect, sys
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import PIL
import math
from tqdm import tqdm
from torch import nn
from torch.nn import functional as F
from dataset import NormalizedDataset, IncompleteDataset, get_stats, get_loaders

In [3]:
white = 255
black = 0

def inverse_color(img):
    return PIL.Image.eval(img, lambda val: 255 - val)

def fix_background_color_bug(img):
    colors = sorted(img.getcolors(), key=lambda pair: pair[0], reverse=True)
    replace_color = colors[0][1]
    remove_color = colors[2][1] if colors[2][1] < colors[1][1] else colors[1][1]

    data = np.array(img)
    data[data == remove_color] = replace_color
    return PIL.Image.fromarray(data)

def remove_background(img):
    return PIL.Image.eval(img, lambda val: 0 if val < (256/2) else val)

def to_binary(img):
    return PIL.Image.eval(img, lambda val: 255 if val < (256/2) else 0)

In [4]:
binary_image = False

## Rotation, Brightness and Resizing
random_transforms = [
    transforms.RandomRotation(360, fill=black),
    transforms.RandomAffine(0, shear=15, scale=(0.8, 0.95), translate=(.03, .03)),
]

format_transforms = [
    transforms.Resize((25, 25)),
]

if not binary_image:
    random_transforms.append(transforms.ColorJitter(brightness=(0.9, 1), contrast=(0.7, 1)))
else:
    format_transforms.append(transforms.Lambda(to_binary))

all_transforms = transforms.Compose(random_transforms + format_transforms)
format_transforms = transforms.Compose(format_transforms)

## Adding Grayscale + Inverse color to operators
operators_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Lambda(inverse_color),
    all_transforms,
])

video_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Lambda(inverse_color),
    transforms.Lambda(remove_background),
    format_transforms,
])

operators_tensor_transform = transforms.Compose([
    operators_transform,
    transforms.ToTensor(),
])

video_tensor_transform = transforms.Compose([
    video_transform,
    transforms.ToTensor(),
])

In [6]:
path = "../video_dataset/op"
op_video_dataset = datasets.ImageFolder(root=path, transform=video_tensor_transform)
datasets.ImageFolder(root=path, transform=video_transform)[2][0]

In [9]:
# Operators Dataset
path = '../operators'
operators_dataset = datasets.ImageFolder(root=path, transform=operators_tensor_transform)

# Show exemple image
datasets.ImageFolder(root=path, transform=operators_transform)[4][0]

In [10]:
# Computing mean and std
operators_mean, operators_std = get_stats(operators_dataset)

# Normalizing
operators_dataset = NormalizedDataset(operators_dataset)

In [11]:
# Creating a dataset and loader with only the video images
op_incomplete_dataset = IncompleteDataset(op_video_dataset, operators_dataset.classes, operators_mean, operators_std)
test_loader = torch.utils.data.DataLoader(op_incomplete_dataset, batch_size=100)

In [12]:
# Training set
train_loader = torch.utils.data.DataLoader(operators_dataset, batch_size=100)
len(train_loader), len(test_loader)

(1, 1)

In [13]:
# from keras.regularizers import L1L2
# from keras.layers import Conv2D
# Creating a Net class object, which consists of 2 convolutional layers, max-pool layers and fully-connected layers
class Conv_Net(nn.Module):
    
    def __init__(self, nb_hidden=25):        
        super(Conv_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)  # the first convolutional layer, which processes the input image
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)  # the second convolutional layer, which gets the max-pooled set
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3)  # the second convolutional layer, which gets the max-pooled set

        self.fc1 = nn.Linear(64, nb_hidden)  # the first fully-connected layer, which gets flattened max-pooled set
        self.fc2 = nn.Linear(nb_hidden, 5)  # the second fully-connected layer that outputs the result

    # Creating the forward pass
    def forward(self, x):
        
        # The first two layers
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        
#         x = nn.Dropout(p=0.5)(x)
        
        # The second two layers
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) 
        
#         x = nn.Dropout(p=0.5)(x)
        
        x = F.relu(F.max_pool2d(self.conv3(x), kernel_size=2)) 

        # Flattening the data set for fully-connected layer
        x = x.view(x.size(0), -1)
    
        # The first fully-connected layer
        x = F.relu(self.fc1(x))
        
        x = nn.Dropout(p=0.3)(x)
        
        # The second full-connected layer
        x = self.fc2(x)
        
        return x

In [ ]:
%%time

model = Conv_Net()

losses = []
test_losses = []

# Defining the optimizer for GD
lr = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr = lr) 

# Defining the criterion to calculate loss
criterion = nn.CrossEntropyLoss()

# Train the model
nb_epochs = 50000
accs = []
best_acc = 0

for e in range(nb_epochs):
    # Train the input dataset by dividing it into mini_batch_size small datasets
    acc = 0
    model.train()
    for train_input, train_target in train_loader:

        # Model computations
        output = model(train_input)
        loss = criterion(output, train_target) 
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        
        tmp_acc = 0
        for tensor, target in zip(output, train_target):
            _, index = tensor.max(0)
            if index == target:
                tmp_acc += 1
                
        tmp_acc /= len(output)
        acc += tmp_acc 
        
    acc /= len(train_loader)
    
    should_print = e%101 == 0
    if should_print:
        print('%dth epoch, test_acc: %f' % (e+1, acc), end="")
    losses.append(loss)
    
    # Train the input dataset by dividing it into mini_batch_size small datasets
    acc = 0
    model.eval()
    for test_input, test_target in test_loader:
        
        output = model(test_input)
        loss = criterion(output, test_target) 
        
        tmp_acc = 0
        for tensor, target in zip(output, test_target):
            _, index = tensor.max(0)
            if index == target:
                tmp_acc += 1
            elif should_print:
                print(", {} != {}".format(index, target), end="")
                
        tmp_acc /= len(output)
        acc += tmp_acc 
                
    acc /= len(test_loader)
    accs.append(acc)
    
    if should_print:
        current_acc = sum(accs) / len(accs)
        if current_acc > best_acc:
            best_acc = current_acc
            torch.save(model.state_dict(), "operator_model")
            
        print(", test acc: %f" % current_acc)
        accs = []
    test_losses.append(loss)

1th epoch, test_acc: 0.000000, 2 != 0, 2 != 1, 2 != 3, 0 != 4, test acc: 0.000000
102th epoch, test_acc: 0.625000, 0 != 1, 3 != 4, test acc: 0.351485
203th epoch, test_acc: 0.750000, 0 != 1, 3 != 4, test acc: 0.576733
304th epoch, test_acc: 0.750000, test acc: 0.844059
405th epoch, test_acc: 0.875000, 3 != 4, test acc: 0.898515
506th epoch, test_acc: 0.875000, test acc: 0.920792
607th epoch, test_acc: 1.000000, test acc: 0.957921


In [147]:
model = Conv_Net()
model.load_state_dict(torch.load("operator_model"))
criterion = nn.CrossEntropyLoss()

test_loader = torch.utils.data.DataLoader(op_incomplete_dataset, batch_size=100)
acc = 0
for test_input, test_target in test_loader:

    output = model(test_input)
    loss = criterion(output, test_target) 

    tmp_acc = 0
    for tensor, target in zip(output, test_target):
        _, index = tensor.max(0)
        if index == target:
            tmp_acc += 1
        elif True:
            print(", {} != {}".format(index, target), end="")

    tmp_acc /= len(output)
    acc += tmp_acc 

acc /= len(test_loader)


In [148]:
acc

1.0